In [1]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
import time
from skimage.measure import label, regionprops, regionprops_table
import pandas
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

In [2]:
def fft_padding(img, kernel):
  p = img.shape[0] + kernel.shape[0] - 1
  q = img.shape[1] + kernel.shape[1] - 1
  new_img = np.zeros((p, q))
  new_img[0:img.shape[0], 0:img.shape[1]]=img
  new_kernel = np.zeros((p, q))
  new_kernel[0:kernel.shape[0], 0:kernel.shape[1]]=kernel
  return new_img, new_kernel


def convolve(img, kernel, iter):
  new_img, new_kernel = fft_padding(img, kernel)
  fft_img=np.fft.fft2(new_img)
  fft_kernel=np.fft.fft2(new_kernel)
  convolved_img=fft_img * (fft_kernel ** iter)
  convolved_img=np.round(np.fft.ifft2(convolved_img).real)
  return convolved_img


def processing(img):
  #noise reduction
  mean_filter = 1/49 * np.ones((7,7))
  smooth_img = convolve(img, mean_filter, 2)

  #edge detection
  sobel_x = np.array([[-5, 0, 5], [-5, 0, 5], [-5, 0, 5]])
  sobel_y = np.array([[-5, -5, -5], [0, 0, 0], [5, 5, 5]])
  sobel_x = convolve(smooth_img, sobel_x, 1)
  sobel_y = convolve(smooth_img, sobel_y, 1)
  gradient = np.sqrt(np.square(sobel_x) + np.square(sobel_y))
  thresh, ret=cv2.threshold(gradient, 50, 255, cv2.THRESH_BINARY)
  ret=cv2.dilate(ret, (5,5), iterations=2)
  ret=cv2.erode(ret, (9,9), iterations=2)
  return ret[8:, 8:]

In [3]:
dir_files = os.listdir('/content')
extension ='.sav'
sf_files = [file for file in dir_files if os.path.splitext(file)[1] == extension][0]
rf_model = pickle.load(open(f'/content/{sf_files}', 'rb'))

In [4]:
if not os.path.exists('measured_imgs'):
  os.mkdir('measured_imgs')

if not os.path.exists('results'):
  os.mkdir('results')

properties = ['area','area_bbox',	'area_convex',	'area_filled',\
              'axis_major_length',	'axis_minor_length']

code={0: 'no_sphere', 1: 'sphere'}

if 'good_imgs' not in locals():
  good_imgs = os.listdir(os.getcwd())

for j in good_imgs:
  measurements = pandas.DataFrame()
  if j.endswith('.tif'):
    img=cv2.imread(j, 0)
    color_img=cv2.imread(j)
    processed_img = processing(img)
    contours, hierarchy = cv2.findContours(processed_img.astype('uint8'),\
                                           cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for contour in contours:
      blank = np.zeros((processed_img.shape))
      blank = cv2.drawContours(blank.astype('uint8'), contour, -1, (255, 255, 255), 3)
      label_img = label(blank, connectivity=2)
      props = regionprops_table(label_img, properties=properties)
      answer = rf_model.predict(pandas.DataFrame(props))[0]
      if answer == 1 or answer == '1':
        measurements=pandas.concat([measurements, pandas.DataFrame(props)])
        x, y, w, h = cv2.boundingRect(contour)
        color_img=cv2.rectangle(color_img.astype('uint8'),(x,y), (x+w,y+h), (36,255,12), 1)
    cv2_imshow(color_img)
    time.sleep(1)
    measurements.to_csv(f'/content/results/{j[:-4]}_measurements.csv')
    cv2.imwrite(f'/content/measured_imgs/{j[:-4]}.jpg', color_img)
    clear_output()

In [ ]:
!zip -r /content/measured_imgs.zip /content/measured_imgs
!zip -r /content/results.zip /content/results
from google.colab import files
files.download("/content/measured_imgs.zip")
files.download("/content/results.zip")